# Descenso al fondo de un cráter en Marte 

- Diseño de agentes inteligentes
- TC2032.101
- Profesor: Juan Emanuel Ledesma Martínez
- Equipo 7:

| Alumno | Matrícula |
| ---- | ---- |
| Juan Pablo Echeagaray González | A00830646 |


Fecha: 13 de marzo del 2022

## Entregable

Se teoriza que en el fondo de algunos de los cráteres de la superficie marciana es posible encontrar agua y otros elementos interesantes que podrían responder a la pregunta de si en Marte alguna vez hubo vida o si la hay en estos momentos. Por ello, es interesante ver la posibilidad de que alguno de los Rover exploradores logre navegar de manera segura hasta estos cráteres y llegue al fondo de los mismos.

Considere que hay un grupo de ingenieros que desean determinar el tipo de explorador que podría llevar a cabo la tarea de descender por un cráter de manera exitosa. Para ello, quieren realizar una serie de simulaciones utilizando como ejemplo el siguiente cráter de la superficie marciana extraído del sitio de [HiRISE](https://www.uahirise.org/dtm/).

![](img/2022-03-14-15-02-24.png)

![](img/2022-03-14-15-02-38.png)

![](img/2022-03-14-15-02-51.png)

La información mostrada con anterioridad se puede representar con una imagen de falso color como la que se muestra a continuación. El color de cada pixel de la imagen representa la altura en dicha posición.

![](img/2022-03-14-15-03-12.png)

Los datos de este cráter tienen dimensiones de 7163 pixeles x 10770 pixeles. Cada pixel representa un área de 1.0045 metros x 1.0045 metros. Los datos mostrados representan la profundidad medida con instrumentos satelitales para cada pixel. El nivel más bajo corresponde a los 0 metros, y se representa con rojo. Los datos mostrados se pueden descargar aquí [Descargar aquí](https://experiencia21.tec.mx/courses/241470/files/78461918?wrap=1) . Para escalar los datos y generar una matriz de numpy utilicen este script [Descargar script](https://experiencia21.tec.mx/courses/241470/files/78461984/download?download_frd=1) . En los datos escalados, cada píxel representa un área de 10.045 metros x 10.045 metros.

El robot explorador que se desea enviar sólo es capaz de utilizar información de su alrededor para tomar decisiones. Es decir, sólo conocerá la profundidad en su vecindad. Por ello, los ingenieros decidieron que en la simulaciones, si el explorador se encuentra en cierto pixel de la imagen, sólo será capaz de conocer la profundidad en los ocho pixeles vecinos. Además, para evitar daños en el robot, no se permite que el explorador se mueva a pixeles con una diferencia de altura mayor a 0.5 metros de la posición en donde se encuentre.

Para llevar a cabo las simulaciones del descenso, como una primera aproximación, se desea probar algoritmos de búsqueda local tales como búsqueda codiciosa y recocido simulado como mecanismos de toma de decisiones del robot. Para ello, realicen lo siguiente.

### Búsqueda codiciosa

Implementen un algoritmo de búsqueda codiciosa o voraz, con el cual, dado una posición inicial en la imagen del cráter, el explorador busque un camino de descenso para llegar a la parte del cráter más profunda. Una manera de implementar dicho algoritmo sería la siguiente:

- Dado el pixel donde se encuentre el explorador, verifique las profundidades de los ocho pixeles vecinos de dicha posición.
- Escoja el pixel que tenga la menor profundidad posible, pero que la diferencia entre la altura actual y la de la posición vecina no sea mayor a 2.0 metros. Si esta profundidad es mejor que la actual, mueva el explorador a dicha posición.
- Repita lo mismo hasta que el explorador no sea capaz de encontrar una mejor posición.

Prueben su algoritmo en la posición x = 3350 metros y = 5800 metros. ¿Hasta donde es capaz el explorador de llegar? Prueben su algoritmo con otras cinco posiciones cercanas y lejanas al fondo del cráter. ¿Qué tan bueno es el algoritmo para llegar al fondo del cráter es los casos probados?

### Búsqueda con recocido simulado

Utilicen el algoritmo de recocido simulado para encontrar una ruta que lleve al fondo del cráter con el explorador. Cada iteración del algoritmo la pueden implementar de la siguiente manera:

- Dado el pixel donde se encuentre el explorador, seleccione uno de los pixeles vecinos de manera aleatoria, con la condición que la diferencia de alturas entre la posición actual y la del vecino no sea mayor a 0.5 metros.
- Si la nueva posición es mejor que la anterior, mueva el explorador a la posición anterior. De lo contrario, pruebe con la regla de recocido simulado si se acepta dicha nueva posición

Realicen las mismas pruebas llevadas a cabo búsqueda codiciosa. ¿Qué algoritmo logra llegar más profundo en el cráter? ¿Recomendarían a los ingenieros del robot utilizar alguno de estos algoritmos?

Al final, cada integrante del equipo deberá incluir sus contribuciones y conclusiones.

### Dependencias

In [48]:
from simpleai.search import SearchProblem
from simpleai.search import greedy, simulated_annealing
import numpy as np
import pandas as pd

from IPython.core.display import HTML
HTML(r"""
<style>
    .output-plaintext, .output-stream, .output {
        font-family: Cascadia Code; # Any monospaced font should work
        line-height: 1.3 !important;
        font-size: 14px !important;
    }
</style>
""")


### Condiciones iniciales

In [49]:
SCALE = 10.0174
crater_map = np.load('crater_map.npy')
NR, NC = crater_map.shape

# Start position
x0, y0 = 3350, 5800
MAX_DIFF = 0.5
UNTRAVERSABLE = -1
STEP_SIZE = 1


### Funciones auxiliares

In [58]:
def pos_to_index(pos: list[int]) -> list[int]:
    nr = NR - round(pos[0] / SCALE)
    nc = round(pos[1] / SCALE)
    return [nr, nc]


def coord_to_state(coord: list[int]) -> str:
    return f'{coord[0]},{coord[1]}'


def state_to_coord(state: str) -> list[int]:
    return tuple([int(x) for x in state.split(',')])


def report(result):
    print(f'Found path with length {len(result.path())}')
    print(f'Path: {result.path()}')
    print(f'Cost: {result.cost}')


def distance(pos1, pos2):
    return np.sqrt((pos1[0] - pos2[0]) ** 2 + (pos1[1] - pos2[1]) ** 2)


def get_indices_of_k_smallest(arr, k):
    # https://stackoverflow.com/a/51523226/15180258
    idx = np.argpartition(arr.ravel(), k)
    # return tuple(np.array(np.unravel_index(idx, arr.shape))[:, range(min(k, 0), max(k, 0))])
    # if you want it in a list of indices . . . 
    return np.array(np.unravel_index(idx, arr.shape))[:, range(k)].transpose().tolist()


def check_surroundings(map, pos, delta):
    x, y = pos
    surroundings = map[x - delta:x + delta + 1, y - delta:y + delta + 1]
    return np.round(surroundings, decimals=2)
    

## Punto más bajo

Una aproximación a la solución de este probolema sería que nosotros mismos encontremos el punto más bajo del cráter, con esta información reduciríamos el problema de optimización a un problema de búsqueda. Para esta entrega pensamos usar la altura de este punto como un valor de referencia, con el cuál podremos medir el desempeño general de las implementaciones que hagamos.


In [59]:
x_min, y_min = np.unravel_index(np.argmin(np.sqrt(crater_map * crater_map), axis=None), crater_map.shape)
print(f'Min height: {crater_map[x_min, y_min]} at ({x_min}, {y_min})')


Min height: 0.09667470703125218 at (58, 66)


Hay que destacar que las _coordenadas_ que regresa la directiva anterior son índices de la matriz, no coordenadas $(x, y)$ de verdad.

### Analizando los alrededores

In [74]:
change = 4
check_surroundings(crater_map, (x_min, y_min), change)

array([[-1.0000e+00, -1.0000e+00, -1.0000e+00, -1.0000e+00, -1.0000e+00,
        -1.0000e+00, -1.0000e+00, -1.0000e+00, -1.0000e+00],
       [-1.0000e+00, -1.0000e+00, -1.0000e+00, -1.0000e+00, -1.0000e+00,
        -1.0000e+00, -1.0000e+00, -1.0000e+00, -1.0000e+00],
       [-1.0000e+00, -1.0000e+00, -1.0000e+00, -1.0000e+00, -1.0000e+00,
        -1.0000e+00, -1.0000e+00, -1.0000e+00, -1.0000e+00],
       [-1.0000e+00, -1.0000e+00, -1.0000e+00, -1.0000e+00, -1.0000e+00,
        -1.0000e+00, -1.0000e+00, -1.0000e+00, -1.0000e+00],
       [-1.0000e+00, -1.0000e+00, -1.0000e+00, -1.0000e+00,  1.0000e-01,
         9.9800e+00,  1.9840e+01,  2.6380e+01,  3.4020e+01],
       [ 9.9400e+00,  3.7390e+01,  6.7060e+01,  9.4540e+01,  1.0859e+02,
         1.0834e+02,  1.0829e+02,  1.0819e+02,  1.0812e+02],
       [ 1.0838e+02,  1.0867e+02,  1.0874e+02,  1.0850e+02,  1.0807e+02,
         1.0774e+02,  1.0758e+02,  1.0751e+02,  1.0748e+02],
       [ 1.0792e+02,  1.0860e+02,  1.0933e+02,  1.0920e+02,  1

El punto asociado con la menor altura será inalcanzable para el agente si es que aplicamos la condición de que la máxima diferenccia de alturas sea de 0.5 metros.

In [61]:
k = 10
small_idx = get_indices_of_k_smallest(np.sqrt(crater_map * crater_map), k)
heights = [crater_map[x, y] for x, y in small_idx]

In [62]:
print(f'Top {k} Smallest heights')
for idx in small_idx:
    print(f'{crater_map[idx[0], idx[1]]} at ({idx[0]}, {idx[1]})')

Top 10 Smallest heights
0.36616933593750217 at (213, 9)
0.3613402343750022 at (24, 588)
0.35244375000000217 at (7, 504)
0.3410302734377183 at (583, 295)
0.2324828125000022 at (69, 21)
0.09667470703125218 at (58, 66)
0.26782226562521827 at (584, 294)
0.2879808593750022 at (70, 17)
0.1605760742187522 at (53, 130)
0.36660390625000217 at (48, 592)


In [66]:
for idx in small_idx:
    print(f'''
{crater_map[idx[0], idx[1]]} at ({idx[0]}, {idx[1]})
{check_surroundings(crater_map, idx, 3)}
----------------------------------------''')


0.36616933593750217 at (213, 9)
[[ -1.    -1.    -1.    31.63 134.41 133.86 133.79]
 [ -1.    -1.    -1.    23.52 135.12 134.81 134.42]
 [ -1.    -1.    -1.    12.65 135.46 135.33 135.06]
 [ -1.    -1.    -1.     0.37 135.65 135.48 135.54]
 [ -1.    -1.    -1.    -1.   128.93 135.68 135.91]
 [ -1.    -1.    -1.    -1.   119.33 135.73 135.81]
 [ -1.    -1.    -1.    -1.   100.26 135.81 135.77]]
----------------------------------------

0.3613402343750022 at (24, 588)
[[135.59 135.55  60.44  -1.    -1.    -1.    -1.  ]
 [135.91 135.72  90.47  -1.    -1.    -1.    -1.  ]
 [136.14 135.41 113.28  -1.    -1.    -1.    -1.  ]
 [135.29 134.83 131.87   0.36  -1.    -1.    -1.  ]
 [134.69 134.48 134.26  19.31  -1.    -1.    -1.  ]
 [134.27 133.86 134.29  40.96  -1.    -1.    -1.  ]
 [133.43 134.06 134.37  61.4   -1.    -1.    -1.  ]]
----------------------------------------

0.35244375000000217 at (7, 504)
[[ -1.    -1.    -1.    -1.    -1.    -1.    -1.  ]
 [ -1.    -1.    -1.    -1.    -1.   

## Implementación

In [182]:
class mars_descent(SearchProblem):

    def __init__(self, initial_state: str, mars_map: np.array, max_height: float, not_traversable: int, step_size: int, max_iter: int):

        self.mars_map = mars_map
        self.initial_state = initial_state
        self.max_height = max_height
        self.not_traversable = not_traversable
        self.step_size = step_size
        self.max_iter = max_iter
        self.n = 0
        print("Initial state: ", self.initial_state)
        
        SearchProblem.__init__(self, initial_state)

    def actions(self, state):
        mars_map = self.mars_map
        actions = []

        pos_directions = ['U', 'D', 'L', 'R', 'UL', 'UR', 'DL', 'DR']
        x0, y0 = state_to_coord(state)
        current_height = mars_map[x0, y0]

        for action in pos_directions:
            x, y = state_to_coord(self.result(state, action))
            next_height = mars_map[x, y]
            height_difference = abs(current_height - next_height)

            if next_height != self.not_traversable:
                if height_difference <= self.max_height:
                    if next_height <= current_height:
                        # print(f'{action}: {current_height} -> {next_height} ({next_height - current_height})')
                        actions.append(action)
                    # # print(f'[{x}, {y}], action: {action}, height_difference: {height_difference}')
                    # actions.append(action)
    
        return actions

    def result(self, state, action):
        x, y = state_to_coord(state)
        step_size = self.step_size

        if action.count('U'):
            x -= step_size
        elif action.count('D'):
            x += step_size
        elif action.count('L'):
            y -= step_size
        elif action.count('R'):
            y += step_size
        elif action.count('UL'):
            x -= step_size
            y -= step_size
        elif action.count('UR'):
            x -= step_size
            y += step_size
        elif action.count('DL'):
            x += step_size
            y -= step_size
        elif action.count('DR'):
            x += step_size
            y += step_size
        else:
            print('No se encontró la acción')
        
        new_state = coord_to_state([x, y])
        # print(action)
        # print(new_state, self.mars_map[state_to_coord(new_state)])
        return new_state

    def is_goal(self, state):
        # check if the current height is smallest possible
        # For reference, the lowest height in the array is around ~0.09
        self.n += 1
        # print(self.n)
        print(f'Iteracion: {self.n}, Posición: {state}, Altura: {self.mars_map[state_to_coord(state)]}')
        tol = 1
        return self.n >= self.max_iter
        # return self.mars_map[state_to_coord(state)] < tol


    def value(self, state):
        # Required by simmulated annealing
        # Used for the optimization process
        return self.mars_map[state_to_coord(state)]


    def cost(self, state, action, state2):
        alpha, beta = 1, 0

        x0, y0 = state_to_coord(state)
        xn, yn = state_to_coord(state2)

        distance = np.sqrt((x0 - xn)**2 + (y0 - yn)**2)
        height_difference = abs(self.mars_map[xn, yn] - self.mars_map[x0, y0])

        return self.mars_map[xn, yn]

    def heuristic(self, state):
        # Minimize height ? We already know the position with the lowest height
        x, y = state_to_coord(state)
        current_height = self.mars_map[x, y]
        return current_height

In [183]:
START_STATE = coord_to_state(pos_to_index([x0, y0]))

In [184]:
crater_problem = mars_descent(START_STATE, crater_map, MAX_DIFF, UNTRAVERSABLE, STEP_SIZE, 1_000_000)

Initial state:  743,579


### Búsqueda voraz

In [185]:
greedy_res = greedy(crater_problem, graph_search=True)

Iteracion: 1, Posición: 743,579, Altura: 151.42256835937522
Iteracion: 2, Posición: 744,579, Altura: 151.29061523437522
Iteracion: 3, Posición: 745,579, Altura: 151.07991455078147
Iteracion: 4, Posición: 745,578, Altura: 150.99151123046897
Iteracion: 5, Posición: 746,578, Altura: 150.8457763671877
Iteracion: 6, Posición: 746,577, Altura: 150.83238037109396
Iteracion: 7, Posición: 746,576, Altura: 150.80159423828147
Iteracion: 8, Posición: 745,576, Altura: 150.75978759765647
Iteracion: 9, Posición: 747,576, Altura: 150.77514404296898
Iteracion: 10, Posición: 748,576, Altura: 150.73098876953148
Iteracion: 11, Posición: 749,576, Altura: 150.71000732421896
Iteracion: 12, Posición: 749,577, Altura: 150.65796142578148
Iteracion: 13, Posición: 750,576, Altura: 150.68088134765648
Iteracion: 14, Posición: 750,577, Altura: 150.65835937500023
Iteracion: 15, Posición: 751,577, Altura: 150.6321533203127
Iteracion: 16, Posición: 745,577, Altura: 150.87178955078147
Iteracion: 17, Posición: 746,579, A

In [181]:
greedy_res.path()

AttributeError: 'NoneType' object has no attribute 'path'

In [168]:
greedy_res.cost

AttributeError: 'NoneType' object has no attribute 'cost'

In [169]:
greedy_res.path()[-1][1]

AttributeError: 'NoneType' object has no attribute 'path'

In [16]:
crater_map[state_to_coord(greedy_res.path()[-1][1])]


0.35244375000000217

In [17]:
state_to_coord(greedy_res.path()[-1][1])

(7, 504)

Este comportamiento es el esperado? Cómo hacer que el algoritmo se detenga solamente cuando esté en una altura menor a la anterior? Cómo codificas eso en esta librería?

### Recocido simulado

## Contribuciones

- Juan Pablo Echeagaray González

## Conclusiones

- Juan Pablo Echeagaray González